In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import numpy as np
import requests
import os.path
import time
import re

In [2]:
start=time.time() #Track the extraction time
start_day=pd.to_datetime('2010-01-01') #Test Ranges
end_day=pd.to_datetime('2022-10-25')

url = 'https://www.estesparkweather.net/archive_reports.php?date={}'
dates = pd.period_range(start_day, end_day, freq='M') #test date range
final_df = pd.DataFrame(columns=['Average temperature (°F)', 'Average humidity (%)',
                             'Average dewpoint (°F)', 'Average barometer (in)',
                             'Average windspeed (mph)', 'Average gustspeed (mph)',
                             'Average direction (°deg)', 'Rainfall for month (in)',
                             'Rainfall for year (in)', 'Maximum rain per minute',
                             'Maximum temperature (°F)', 'Minimum temperature (°F)',
                             'Maximum humidity (%)', 'Minimum humidity (%)', 'Maximum pressure',
                             'Minimum pressure', 'Maximum windspeed (mph)',
                             'Maximum gust speed (mph)', 'Maximum heat index (°F)', 'Date'])

In [3]:
for i in dates:
    if i.month < 10:
        date = str(i.year) + str(0) + str(i.month)
    else:
        date = str(i.year) + str(i.month)
    #print(date)
    link = url.format(date)
    data=requests.get(link)
    d=data.text
    soup=bs(d, features="html.parser")
    tables=soup.find_all("table")
    #elem=table.find_all("")
    month=[]
    for k in range(len(tables)):
        elem=tables[k].find_all("td")
        letter=elem[0].text
        if "Average" == letter[:7]:
            break
        else:
            date_str = re.search("[A-Z][a-z]+ [0-9]+", elem[0].text).group() + " " + str(i.year)
            dt = pd.to_datetime(date_str)
            #print(dt)
            if int(i.month)==int(end_day.strftime("%m")) and (int(dt.strftime('%d'))==(int(end_day.strftime('%d')) + 1)) and (int(i.year)==int(end_day.strftime("%Y"))):
                break
            else:
                elem=elem[1:]
                day=[]
                for j in range(1,len(elem),2):
                    day.append(elem[j].text)
                day.append(dt)
                month.append(day)
    df=pd.DataFrame(month, columns=['Average temperature (°F)', 'Average humidity (%)',
                             'Average dewpoint (°F)', 'Average barometer (in)',
                             'Average windspeed (mph)', 'Average gustspeed (mph)',
                             'Average direction (°deg)', 'Rainfall for month (in)',
                             'Rainfall for year (in)', 'Maximum rain per minute',
                             'Maximum temperature (°F)', 'Minimum temperature (°F)',
                             'Maximum humidity (%)', 'Minimum humidity (%)', 'Maximum pressure',
                             'Minimum pressure', 'Maximum windspeed (mph)',
                             'Maximum gust speed (mph)', 'Maximum heat index (°F)', 'Date'])
    # print(df.info())
    final_df=pd.concat([final_df,df], ignore_index=True)
final_df.set_index("Date", inplace=True)
if os.path.exists("Weather_Data_2010-2022.csv"):
    final_df.to_csv('Weather_Data_2010-2022.csv', mode="a", header=False) # action: retain same index
else:
    final_df.to_csv('Weather_Data_2010-2022.csv')
end=time.time()
print("Time taken:",end-start)


Time taken: 102.67043948173523


In [4]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4353 entries, 2010-01-01 to 2022-10-25
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Average temperature (°F)  4353 non-null   object
 1   Average humidity (%)      4353 non-null   object
 2   Average dewpoint (°F)     4353 non-null   object
 3   Average barometer (in)    4353 non-null   object
 4   Average windspeed (mph)   4353 non-null   object
 5   Average gustspeed (mph)   4353 non-null   object
 6   Average direction (°deg)  4353 non-null   object
 7   Rainfall for month (in)   4353 non-null   object
 8   Rainfall for year (in)    4353 non-null   object
 9   Maximum rain per minute   4353 non-null   object
 10  Maximum temperature (°F)  4353 non-null   object
 11  Minimum temperature (°F)  4353 non-null   object
 12  Maximum humidity (%)      4353 non-null   object
 13  Minimum humidity (%)      4353 non-null   object
 14  Maximu

In [5]:
final_df

,Average temperature (°F),Average humidity (%),Average dewpoint (°F),Average barometer (in),Average windspeed (mph),Average gustspeed (mph),Average direction (°deg),Rainfall for month (in),Rainfall for year (in),Maximum rain per minute,Maximum temperature (°F),Minimum temperature (°F),Maximum humidity (%),Minimum humidity (%),Maximum pressure,Minimum pressure,Maximum windspeed (mph),Maximum gust speed (mph),Maximum heat index (°F)
Date,,,,,,,,,,,,,,,,,,,
2010-01-01,32.1�F,49%,15.2�F,30.2 in.,14.6 mph,19.7 mph,297� (WNW),0.00 in.,0.00 in.,0.00 in. on day 01 at time 00:00,40.0�F on day 01 at time 13:15,22.1�F on day 01 at time 01:12,61% on day 01 at time 20:54,41% on day 01 at time 13:16,30.330 in. on day 01 at time 07:53,30.034 in. on day 01 at time 00:00,34.5 mph on day 01 at time 03:23,51 mph from 315 �( NW) on day 01 at time 03:08,40.0�F on day 01 at time 13:15
2010-01-02,32.1�F,50%,15.5�F,30.1 in.,8.8 mph,12.2 mph,306� ( NW),0.00 in.,0.00 in.,0.00 in. on day 02 at time 23:57,39.1�F on day 02 at time 11:11,22.0�F on day 02 at time 23:57,77% on day 02 at time 11:50,39% on day 02 at time 13:26,30.292 in. on day 02 at time 23:38,29.937 in. on day 02 at time 13:23,23.0 mph on day 02 at time 13:00,35 mph from 315 �( NW) on day 02 at time 13:15,39.1�F on day 02 at time 11:11
2010-01-03,23.1�F,64%,12.1�F,30.4 in.,2.9 mph,4.4 mph,21� (NNE),0.00 in.,0.00 in.,0.00 in. on day 03 at time 23:57,33.4�F on day 03 at time 10:52,9.2�F on day 03 at time 23:57,85% on day 03 at time 18:53,34% on day 03 at time 11:02,30.549 in. on day 03 at time 21:38,30.237 in. on day 03 at time 03:23,17.3 mph on day 03 at time 07:31,24 mph from 315 �( NW) on day 03 at time 07:28,33.4�F on day 03 at time 10:52
2010-01-04,25.7�F,48%,7.2�F,30.4 in.,4.7 mph,7.0 mph,324� ( NW),0.00 in.,0.00 in.,0.00 in. on day 04 at time 23:57,36.5�F on day 04 at time 12:04,7.6�F on day 04 at time 01:25,82% on day 04 at time 01:33,25% on day 04 at time 12:07,30.585 in. on day 04 at time 01:38,30.107 in. on day 04 at time 23:57,15.0 mph on day 04 at time 13:50,21 mph from 293 �(WNW) on day 04 at time 12:43,36.5�F on day 04 at time 12:04
2010-01-05,34.3�F,51%,17.8�F,30.0 in.,14.2 mph,18.5 mph,300� (WNW),0.00 in.,0.00 in.,0.00 in. on day 05 at time 23:57,40.1�F on day 05 at time 15:09,28.4�F on day 05 at time 03:44,60% on day 05 at time 23:50,43% on day 05 at time 06:51,30.173 in. on day 05 at time 03:23,29.897 in. on day 05 at time 20:38,34.5 mph on day 05 at time 10:03,43 mph from 293 �(WNW) on day 05 at time 10:37,40.1�F on day 05 at time 15:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-21,56.0�F,20%,15.0�F,29.5 in.,10.5 mph,16.7 mph,254� (WSW),0.26 in.,8.57 in.,0.00 in. on day 21 at time 23:59,65.4�F on day 21 at time 14:37,49.3�F on day 21 at time 06:16,27% on day 21 at time 07:02,11% on day 21 at time 16:16,29.691 in. on day 21 at time 00:00,29.312 in. on day 21 at time 23:26,23.0 mph on day 21 at time 12:26,38.0 mph from 150 �(SSE) on day 21 at time 1...,65.4�F on day 21 at time 14:37
2022-10-22,53.7�F,25%,18.5�F,29.2 in.,10.9 mph,16.4 mph,233� ( SW),0.26 in.,8.57 in.,0.00 in. on day 22 at time 23:59,62.1�F on day 22 at time 16:16,40.9�F on day 22 at time 23:59,41% on day 22 at time 23:59,15% on day 22 at time 13:48,29.323 in. on day 22 at time 00:18,29.077 in. on day 22 at time 23:56,23.0 mph on day 22 at time 14:17,35.7 mph from 269 �( W ) on day 22 at time 1...,62.1�F on day 22 at time 16:16
2022-10-23,40.7�F,46%,21.2�F,29.1 in.,7.3 mph,11.3 mph,221� ( SW),0.26 in.,8.57 in.,0.00 in. on day 23 at time 23:59,51.8�F on day 23 at time 11:44,30.4�F on day 23 at time 23:59,65% on day 23 at time 06:14,29% on day 23 at time 15:42,29.559 in. on day 23 at time 23:31,28.966 in. on day 23 at time 07:47,26.5 mph on day 23 at time 14:36,36.8 mph from 300 �(WNW) on day 23 at time 1...,51.8�F on day 23 at time 11:44
